In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from astropy.coordinates import SkyCoord, Galactocentric
import astropy.units as u


# 讀取資料
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1CKdG7vZ8LacDUbQsLGTs8X2tDDWG1UNm/export?format=csv')


#清除空白資料
df_cleaned = df.dropna(subset=['ra', 'dec', 'pmra', 'pmdec', 'RV', 'distance'])
ra, dec, pmra, pmdec, RV, parallax = df_cleaned['ra'].values, df_cleaned['dec'].values, df_cleaned['pmra'].values, df_cleaned['pmdec'].values, df_cleaned['RV'].values, df_cleaned['distance'].values

ra = df_cleaned['ra'].values
dec = df_cleaned['dec'].values
pmra = df_cleaned['pmra'].values
pmdec = df_cleaned['pmdec'].values
RV = df_cleaned['RV'].values
distance = df_cleaned['distance'].values
parallax = 1000 / distance

for col in ['ra', 'dec', 'pmra', 'pmdec', 'RV', 'distance', 'Fe/H']:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')


# 轉換數據為 numpy 陣列
ra, dec = df_cleaned['ra'].values, df_cleaned['dec'].values
pmra, pmdec = df_cleaned['pmra'].values, df_cleaned['pmdec'].values
RV, distance = df_cleaned['RV'].values, df_cleaned['distance'].values



# 轉換到銀心座標系
coords = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, pm_ra_cosdec=pmra*u.mas/u.yr, pm_dec=pmdec*u.mas/u.yr, radial_velocity=RV*u.km/u.s, distance=distance*u.pc, frame='icrs')
galactocentric = coords.transform_to(Galactocentric)


# 提取銀心坐標中的X, Y, Z 位置和速度分量
X, Y, Z = galactocentric.x.to(u.kpc).value, galactocentric.y.to(u.kpc).value, galactocentric.z.to(u.kpc).value
v_x, v_y, v_z = galactocentric.v_x.to(u.km/u.s).value, galactocentric.v_y.to(u.km/u.s).value, galactocentric.v_z.to(u.km/u.s).value

df_cleaned['X'] = X
df_cleaned['Y'] = Y
df_cleaned['Z'] = Z
df_cleaned['v_x'] = v_x
df_cleaned['v_y'] = v_y
df_cleaned['v_z'] = v_z
df_cleaned['distance_to_gc'] = np.sqrt(df_cleaned['X']**2 + df_cleaned['Y']**2 + df_cleaned['Z']**2)
max_range = np.max([np.max(np.abs(X)), np.max(np.abs(Y)), np.max(np.abs(Z))])


# 計算內積
R = np.array([X, Y, Z])
V = np.array([v_x, v_y, v_z])

df_cleaned['r_dot_v'] = np.sum(R * V, axis=0)
r_dot_v= df_cleaned['r_dot_v'].values

norm_R = np.linalg.norm(R)
norm_V = np.linalg.norm(V)


# 計算夾角（單位：弧度）
cos_theta = r_dot_v / (norm_R * norm_V)
theta_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))

df_cleaned['theta_deg'] = np.degrees(theta_rad)

target_angle = 90.0
tolerance = 1.0

lower = target_angle - tolerance
upper = target_angle + tolerance


# 篩選夾角在範圍內的數據
df_cleaned['Is_Orbiting'] = (df_cleaned['theta_deg'] >= lower) & (df_cleaned['theta_deg'] <= upper)


# 定義位置條件和標籤
conditions = [
    (np.sqrt(df_cleaned['X']**2 + df_cleaned['Y']**2 + df_cleaned['Z']**2) < 3),  #核球
    (np.sqrt(df_cleaned['X']**2 + df_cleaned['Y']**2) < 21) & (np.abs(df_cleaned['Z']) <  1.35),  #銀盤
    (np.abs(df_cleaned['Z']) > 1.35) | (np.sqrt(df_cleaned['X']**2 + df_cleaned['Y']**2) > 21),  #銀暈
]
choices = ['Bulge', 'Disk', 'Halo']


# 新增 'Location' 欄位
df_cleaned['Location'] = np.select(conditions, choices, default='Unknown')

bulge = df_cleaned[(df_cleaned['Location'] == 'Bulge') & df_cleaned['X'].notna()]
disk = df_cleaned[(df_cleaned['Location'] == 'Disk') & df_cleaned['X'].notna()]
halo = df_cleaned[(df_cleaned['Location'] == 'Halo') & df_cleaned['X'].notna()]


# 體積計算
r_bulge = 3
r_halo = 38.51
r_disk = 21
h_disk = 1.35
V_bulge = (4 / 3) * np.pi * (r_bulge**3)
V_disk = np.pi * (r_disk**2) * h_disk
V_halo = (4 / 3) * np.pi * (r_halo**3) - V_bulge - V_disk

print("各位置的體積（每 kpc³）：")
print(f"核球（Bulge）：{V_bulge:.4f}")
print(f"銀盤（Disk）：{V_disk:.4f}")
print(f"銀暈（Halo）：{V_halo:.4f}")


# 星團數量
location_counts = df_cleaned['Location'].value_counts()

print("數量：")
for location, count in location_counts.items():
    print(f"{location}: {count} 個星團")


# 密度計算
density_bulge = location_counts.get('Bulge', 0) / V_bulge
density_disk = location_counts.get('Disk', 0) / V_disk
density_halo = location_counts.get('Halo', 0) / V_halo


# 結果列印
print("各位置的星團密度（每 kpc³）：")
print(f"核球（Bulge）：{density_bulge:.4f}")
print(f"銀盤（Disk）：{density_disk:.4f}")
print(f"銀暈（Halo）：{density_halo:.10f}")


df_cleaned['hover_text'] = df_cleaned.apply(lambda row:
        f"Name: {row['Name']}<br>" +
        f"[Fe/H]: {row['Fe/H']:.2f}<br>" +
        f"Distance to GC: {row['distance_to_gc']:.2f} kpc<br>" +
        f"Location: {row['Location']}", axis=1)


bulge = df_cleaned[df_cleaned['Location'] == 'Bulge'].copy()
disk = df_cleaned[df_cleaned['Location'] == 'Disk'].copy()
halo = df_cleaned[df_cleaned['Location'] == 'Halo'].copy()


# 繪製位置的 3D 圖
fig_location = go.Figure()
fig_location.add_trace(go.Scatter3d(
    x=bulge['X'],
    y=bulge['Y'],
    z=bulge['Z'],
    mode='markers',
    marker=dict(size=4, color='deepskyblue', opacity=0.5),
    text=bulge['hover_text'],
    hoverinfo='text',
    name='Bulge'
))

fig_location.add_trace(go.Scatter3d(
    x=disk['X'],
    y=disk['Y'],
    z=disk['Z'],
    mode='markers',
    marker=dict(size=4, color='orange', opacity=0.5),
    text=disk['hover_text'],
    hoverinfo='text',
    name='Disk'
))

fig_location.add_trace(go.Scatter3d(
    x=halo['X'],
    y=halo['Y'],
    z=halo['Z'],
    mode='markers',
    marker=dict(size=4, color='purple', opacity=0.5),
    text=halo['hover_text'],
    hoverinfo='text',
    name='Halo'
))


earth_x = -8.122
earth_y = 0
earth_z = 0.027
fig_location.add_trace(go.Scatter3d(
    x=[earth_x],
    y=[earth_y],
    z=[earth_z],
    mode='markers+text',
    marker=dict(
        size=10,
        color='navy',
        opacity=1
    ),
    text=["Earth"],
    textposition="bottom center",
    name="Earth"
))


fig_location.add_trace(go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers+text',
    marker=dict(
        size=10,
        color='crimson',
        opacity=1
    ),
    text=["Galactic Core"],
    textposition="bottom center",
    name="Galactic Core",
    legendgroup='Galactic Core',
    showlegend=True
))


fig_location.update_layout(title=' 疏散星團之分布 (銀心座標系)', scene=dict(
        xaxis=dict(range=[-max_range, max_range], title='X (kpc)'),
        yaxis=dict(range=[-max_range, max_range], title='Y (kpc)'),
        zaxis=dict(range=[-max_range, max_range], title='Z (kpc)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)
    ),
    width=1920,
    height=1080,
    margin=dict(r=20, b=10, l=10, t=40),
    showlegend=True,
    legend=dict(
        x=0.7,
        y=0.9,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="lightgray",
        borderwidth=1
    )
)

fig_location.show()

output_csv_path = '疏散星團之分布.csv'
df_cleaned.to_csv(output_csv_path, index=False)

from google.colab import files
files.download('疏散星團之分布.csv')

fig_location.write_html("疏散星團之分布.html")
from google.colab import files
files.download("疏散星團之分布.html")


各位置的體積（每 kpc³）：
核球（Bulge）：113.0973
銀盤（Disk）：1870.3472
銀暈（Halo）：237242.9887
數量：
Disk: 2836 個星團
Halo: 88 個星團
Bulge: 72 個星團
各位置的星團密度（每 kpc³）：
核球（Bulge）：0.6366
銀盤（Disk）：1.5163
銀暈（Halo）：0.0003709277


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>